In [10]:
%load_ext autoreload
%autoreload 2

import os,sys
import transform
import util
from transform import transformFFT
import numpy as np
import re
from scipy.signal import blackmanharris as blackmanharris

db = 'DSD100subset'
feature_path = 'output_folder/features'

os.makedirs(feature_path, exist_ok=True)
assert os.path.isdir(db), "Please input the directory for the DSD100 dataset"
    
mixture_directory = os.path.join(db,'Mixtures')
source_directory = os.path.join(db,'Sources')


def infoAudioScipy(filein):
    sampleRate, audioObj = scipy.io.wavfile.read(filein)
    bitrate = audioObj.dtype
    nsamples = len(audioObj)
    audioObj = None
    return nsamples, sampleRate, bitrate

def readAudioScipy(filein):
    sampleRate, audioObj = scipy.io.wavfile.read(filein)
    bitrate = audioObj.dtype
    try:
        maxv = np.finfo(bitrate).max
    except:
        maxv = np.iinfo(bitrate).max
    return audioObj.astype('float')/maxv, sampleRate, bitrate

def writeAudioScipy(fileout,audio_out,sampleRate,bitrate="int16"):
    maxn = np.iinfo(bitrate).max
    scipy.io.wavfile.write(filename=fileout, rate=sampleRate, data=(audio_out*maxn).astype(bitrate))


#circular shift audio with 'cs'
def circular_shift(audio,min_size,cs=0.1,sampleRate=44100):
    if cs == 0:
        if len(audio) > min_size:
            segment = audio[:min_size]
        else:
            segment = np.zeros(min_size)
            segment[:len(audio)] = audio
    elif cs < 0:
        seg_idx = int(abs(cs*sampleRate))
        segment = np.pad(audio,((0,seg_idx+np.maximum(0,min_size-len(audio)))), mode='constant')
        if len(segment)<(min_size+seg_idx):
            segment = np.pad(segment,((0,min_size+seg_idx - len(segment))), mode='constant')
        segment = segment[seg_idx:min_size+seg_idx]
    else:
        segment = np.pad(audio,((int(cs*sampleRate),0)), mode='constant')
        if len(segment)<min_size:
            segment = np.pad(segment,((0,min_size - len(segment))), mode='constant')
        segment = segment[:min_size]

    return segment


class interpolate:
    def __init__(self,cqt,Ls):
        from scipy.interpolate import interp1d
        self.intp = [interp1d(np.linspace(0,Ls,len(r)),r) for r in cqt]
    def __call__(self,x):
        try:
            len(x)
        except:
            return np.array([i(x) for i in self.intp])
        else:
            return np.array([[i(xi) for i in self.intp] for xi in x])

class interpolate1d:
    def __init__(self,cqt,Ls):
        from scipy.interpolate import interp1d
        self.intp = interp1d(np.linspace(0,Ls,len(cqt)),cqt)
    def __call__(self,x):
        try:
            len(x)
        except:
            return np.array([self.intp(xi) for xi in x],dtype='complex128')
        else:
            return np.array([self.intp(xi) for xi in x],dtype='complex128')

def saveObj(obj,filename):
    with open(filename, 'wb') as output:
        pickle.dump(obj, output, -1)

def loadObj(filename):
    with open(filename, 'rb') as input:
        obj= pickle.load(input)
    return obj

def emptyDir(dirPath):
    fileList = os.listdir(dirPath)
    for fileName in fileList:
        os.remove(os.path.join(dirPath,fileName))

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [9]:
db

'DSD100subset'